In [2]:
from langchain.document_loaders import WebBaseLoader
## Load The data 
URL = [
    "https://en.wikipedia.org/wiki/Titanic",
    "https://www.investopedia.com/terms/s/stockmarket.asp"
]
#load the data
loader = WebBaseLoader(URL)
docs = loader.load()
print("Docs loaded:", len(docs))

USER_AGENT environment variable not set, consider setting it to identify your requests.


Docs loaded: 2


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
chunking = text_splitter.split_documents(docs)

In [4]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

import os
from getpass import getpass

# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_30964\3698268976.py:10: LangChainDeprecationWarning: The class `HuggingFaceInferenceAPIEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpointEmbeddings``.
  embeddings = HuggingFaceInferenceAPIEmbeddings(


In [5]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=chunking,
    embedding=embeddings,
    collection_name="my_collection",
    persist_directory="chroma_db"
)

vectorstore.persist()
print("✅ Vectorstore created successfully")


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_30964\1341209866.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\LENOVO\anaconda3\envs\rag_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


✅ Vectorstore created successfully


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_30964\1341209866.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [6]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":3})

query = "What are the main factors that influence stock prices?"
docs_rel = retriever.get_relevant_documents(query)

for i, doc in enumerate(docs_rel, 1):
    print(f"\n📌 Document {i}:")
    print(doc.page_content[:500])  # show first 500 chars



📌 Document 1:
Why Is the Stock Market So Important?

📌 Document 2:
How Stock Prices Are Determined

📌 Document 3:
Those in the stock market include institutional investors, such as pension funds, mutual funds, insurance companies, and hedge funds, that manage large amounts of money and often have a significant influence over the market since they are trading in


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_30964\2641241717.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_rel = retriever.get_relevant_documents(query)


In [7]:
prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [ ]:
from langchain.llms import HuggingFaceHub

model = HuggingFaceHub(
    repo_id = "tiiuae/falcon-7b-instruct",
,
    model_kwargs={
        "temperature": 0.5,
        "max_new_tokens": 512,
        "max_length": 64
    }
)


In [ ]:
from langchain.chains import RetrievalQA


qa = RetrievalQA.from_chain_type(llm=model,retriever=retriever,chain_type="stuff")
response = qa(prompt)
print(response['result'])